In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time
SFbaseline2018 = pd.read_csv("C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/PersonTrip_SF_Baseline_2018/ASim_BEAM_PersonTrip_SF_Baseline_2018_updated.csv")
#SF_RH_Fleet_2018 = pd.read_csv("C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/PersonTrip_SF_RHFleetsz0125_2018/ASim_BEAM_Merged_SF_RHFleetsz0125_2018.csv")
#SF_RH_Fleet_175_2018 = pd.read_csv("C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/PersonTrip_SF_RHFleetsz175_2018/ASim_BEAM_Merged_SF_RHFleetsz175_2018.csv")

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [88]:
SFbaseline2018.door_to_door_time[SFbaseline2018.door_to_door_time < 0].count()

154

In [6]:
SF_TransitFreq05_2018 = pd.read_csv("C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/PersonTrip_SF_TransitFreq05_2018/ASIM_BEAM_Merged_TransitFreq05_2018.csv")
#SF_TransitFreq2_2018 = pd.read_csv("C:/Shared-Work/Data/CleanData/ASIM_BEAM_Merged/PersonTrip_SF_TransitFreq2_2018/ASIM_BEAM_Merged_TransitFreq2_2018.csv")

In [2]:
# File location on S3 (The address should be updated depending on the version of the code using)
loc_2018_baseline = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-base-20220409/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_0.5-20220228/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_transitFreq2 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_2.0-20220229/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz0125 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/beam/year-2018-iteration-5/ITERS/it.0/"
loc_2018_rhFleetsz175 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.75-20220408/beam/year-2018-iteration-5/ITERS/it.0/"

In [3]:
%%time
# Reading the events file
dtypes = {
    "time": "float32",
    "type": "category",
    "legMode": "category",
    "actType": "category", 
    "primaryFuelLevel": "float64",
    "legMode": "category",
    "chargingPointType":"category",
    "pricingModel":"category",
    "parkingType":"category",
    "mode":"category",
    "personalVehicleAvailable": "category",
    "person": "object",
    "driver": "object",
    "riders": "object"
}

# Use list comprehension to remove the unwanted column in **usecol**
eventsSF = pd.read_csv(loc_2018_baseline + '0.events.csv.gz', compression = 'gzip', dtype = dtypes)

<timed exec>:20: DtypeWarning: Columns (5,7,12,13,14,15,17,23,29,32,44,53,55,56) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 3min 41s


In [4]:
#plansSF = pd.read_csv(loc_2018_tripid + '0.plans.csv.gz', compression = 'gzip', dtype = dtypes)

In [5]:
#plansSF.sort_values(by = "personId").head()

In [6]:
# Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [7]:
eventsSF['scenario'] = "baseline"
eventsSF['year'] = '2018'

In [8]:
# Rename the "mode" column
eventsSF.rename(columns={"mode":"modeBEAM"}, inplace=True) 
# Replace "Work" with "work" in the "actType" column
eventsSF["actType"].replace({"Work": "work"}, inplace=True)

In [9]:
# Remove person = TransitDriver or RidehailDriver because there are no agent information in these rows
eventsSF = eventsSF[~eventsSF.person.str.contains("Agent", na=False)].reset_index(drop=True)

In [10]:
eventsSFCopy = eventsSF.copy()

In [22]:
# shift column 'Name' to first position
first_column = eventsSF.pop('person')
second_column = eventsSF.pop('driver')
third_column = eventsSF.pop('riders')
# insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'person', first_column)
eventsSF.insert(1, 'driver', second_column)
eventsSF.insert(2, 'riders', third_column)

In [11]:
# Adding the IDMerged Column
eventsSF['UniqueID'] = eventsSF['person'] #make a copy of the person column
eventsSF['personID'] = np.where(eventsSF['person'].isin(eventsSF['driver']), eventsSF['person'], np.nan) 
eventsSF['driverID'] = np.where(eventsSF['driver'].isin(eventsSF['person']), eventsSF['driver'], np.nan)

In [12]:
# Merging person and driver ids in one column
eventsSF['IDMerged'] = eventsSF['personID'].combine_first(eventsSF['driverID'])
eventsSF['IDMerged'] = eventsSF['UniqueID'].combine_first(eventsSF['IDMerged'])

In [13]:
# Dropping unused columns
eventsSF = eventsSF.drop(['personID','driverID','UniqueID'], axis=1) 

In [14]:
# Shift column 'Name' to first position
first_column = eventsSF.pop('IDMerged')
# Insert column using insert(position,column_name,first_column) function
eventsSF.insert(0, 'IDMerged', first_column)

In [15]:
%%time
# Split the "riders' column and replicated rows for every rider
eventsSF['riders'] = eventsSF['riders'].str.split(':')
eventsSF = eventsSF.explode('riders')

Wall time: 1min 49s


In [16]:
# Combine riderID with IDMerged
eventsSF['riderID'] = np.where(eventsSF['riders'].isin(eventsSF['person']), eventsSF['riders'], np.nan)
eventsSF['IDMerged'] = eventsSF['riderID'].combine_first(eventsSF['IDMerged'])

In [17]:
# Dropping unused columns
eventsSF = eventsSF.drop(['riderID'], axis=1) 

In [18]:
# Either charging events or just ridehail or transit drivers
IDnan = eventsSF[eventsSF['IDMerged'].isna()]

In [19]:
# Remove driver = TransitDriver or RidehailDriver for IDMerged = NAN because there are no agent information in these rows 
eventsSF = eventsSF[~((eventsSF.driver.str.contains("Agent", na=False))&(eventsSF.IDMerged.isna()))].reset_index(drop=True)

In [20]:
IDnan = eventsSF[eventsSF['IDMerged'].isna()]
IDnan.shape

(3380, 65)

In [21]:
%%time
# Filling NANs in ID related to charging events
eventsSF["chargeID"] = eventsSF.groupby('vehicle')['IDMerged'].transform(lambda x: x.ffill().bfill())

Wall time: 4min 23s


In [22]:
# Combining chargeID with IDMerged so no NANs anymore
eventsSF['IDMerged'] = eventsSF['chargeID'].combine_first(eventsSF['IDMerged'])

In [23]:
# Dropping unused columns
eventsSF = eventsSF.drop(['chargeID'], axis=1) 

In [24]:
print(eventsSF.currentTourMode.unique().tolist())

['hov2_teleportation', nan, 'car', 'car_hov2', 'walk', 'bike', 'hov3_teleportation', 'car_hov3', 'walk_transit', 'ride_hail', 'ride_hail_pooled', 'drive_transit', 'ride_hail_transit', 'bike_transit']


In [26]:
print(eventsSF.modeBEAM.unique().tolist())

['hov2_teleportation', nan, 'car', 'walk', 'car_hov2', 'bike', 'hov3_teleportation', 'car_hov3', 'walk_transit', 'ride_hail', 'ride_hail_pooled', 'drive_transit', 'ride_hail_transit', 'bus', 'tram', 'subway', 'rail', 'cable_car', 'bike_transit']


In [27]:
print(eventsSF.type.unique().tolist())

['ModeChoice', 'actend', 'departure', 'TeleportationEvent', 'arrival', 'actstart', 'PersonEntersVehicle', 'PathTraversal', 'LeavingParkingEvent', 'ParkingEvent', 'PersonCost', 'PersonLeavesVehicle', 'ReserveRideHail', 'Replanning', 'ChargingPlugInEvent', 'RefuelSessionEvent', 'ChargingPlugOutEvent']


In [25]:
%%time
# Change the IDMerged column type to numeric
eventsSF["IDMerged"] = pd.to_numeric(eventsSF.IDMerged)

Wall time: 18.3 s


In [26]:
# Sort by IDMerged and time columns
eventsSF = eventsSF.sort_values(['IDMerged','time']).reset_index(drop=True)

In [27]:
# We assume that the number of passengers is 1 for ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where(eventsSF.driver.str.contains("rideHailAgent", na=False), 'ride_hail' , eventsSF['modeBEAM'])

In [28]:
# Adding teleportation mode to the type = TeleportationEvent row 
eventsSF["modeBEAM_rh"] = np.where(eventsSF['type']=='TeleportationEvent', eventsSF.modeBEAM_rh.fillna(method='ffill'), eventsSF["modeBEAM_rh"])

In [29]:
eventsSFCopy = eventsSF.copy()

In [30]:
eventsSF['modeBEAM_rh_pooled'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_pooled'), 'ride_hail_pooled', np.nan)

In [31]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = np.where((eventsSF['type'] == 'PersonCost') & (eventsSF['modeBEAM'] == 'ride_hail_transit'), 'ride_hail_transit', np.nan)

In [32]:
eventsSF['modeBEAM_rh_pooled'] = eventsSF['modeBEAM_rh_pooled'].shift(+1)

In [33]:
eventsSF['modeBEAM_rh_ride_hail_transit'] = eventsSF['modeBEAM_rh_ride_hail_transit'].shift(+1)

In [34]:
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_pooled'] != 'nan'), eventsSF['modeBEAM_rh_pooled'], eventsSF['modeBEAM_rh'])

In [35]:
# We don't know if ridehail_transit is ride_hail or ride_hail_pooled
eventsSF['modeBEAM_rh'] = np.where((eventsSF['type'] == 'PathTraversal') & (eventsSF['modeBEAM'] == 'car') & (eventsSF['driver'].str.contains("rideHailAgent", na=False)) & (eventsSF['modeBEAM_rh_ride_hail_transit'] != 'nan'), eventsSF['modeBEAM_rh_ride_hail_transit'], eventsSF['modeBEAM_rh'])

In [36]:
# Dropping the temporary columns
eventsSF = eventsSF.drop(['modeBEAM_rh_pooled'], axis=1)
eventsSF = eventsSF.drop(['modeBEAM_rh_ride_hail_transit'], axis=1)

In [37]:
eventsSF.head(200000).to_csv('C:/Shared-Work/Data/CleanData/modeBEAMCHECK.csv', index = False) 

#### Adding new columns

In [38]:
eventsSF['actEndTime'] = np.where(eventsSF['type']=='actend'
                     , eventsSF['time'], np.nan)

In [39]:
eventsSF['actStartTime'] = np.where(eventsSF['type']=='actstart'
                     , eventsSF['time'], np.nan)    

In [40]:
eventsSF['travelTime'] = np.where((eventsSF['type']=='PathTraversal')|(eventsSF['type']=='TeleportationEvent')
                     , eventsSF['arrivalTime'] - eventsSF['departureTime'], np.nan)

In [41]:
eventsSF['travelDistance'] = np.where((eventsSF['type']=='PathTraversal')|((eventsSF['type']=='ModeChoice')&((eventsSF['modeBEAM']=='hov2_teleportation')|(eventsSF['modeBEAM']=='hov3_teleportation'))), eventsSF['length'], np.nan)

In [42]:
eventsSF['length_mode_choice'] = np.where(eventsSF['type']=='ModeChoice', eventsSF['length'], np.nan)

In [43]:
eventsSF['duration_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['travelTime'], np.nan)

In [44]:
eventsSF['distance_walking'] = np.where(eventsSF['modeBEAM']=='walk', eventsSF['travelDistance'], np.nan)

In [45]:
eventsSF['duration_on_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['travelTime'], np.nan)

In [46]:
eventsSF['distance_bike'] = np.where(eventsSF['modeBEAM']=='bike', eventsSF['travelDistance'], np.nan)

In [47]:
eventsSF['duration_in_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), 
                                            eventsSF['travelTime'], np.nan)

In [48]:
eventsSF['distance_ridehail'] = np.where((eventsSF['modeBEAM_rh']=='ride_hail')|(eventsSF['modeBEAM_rh']=='ride_hail_pooled')|(eventsSF['modeBEAM_rh']=='ride_hail_transit'), eventsSF['travelDistance'], np.nan)

In [49]:
eventsSF['duration_in_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation') 
                                              , eventsSF['travelTime'], np.nan)

In [50]:
eventsSF['distance_privateCar'] = np.where((eventsSF['modeBEAM_rh']=='car')|(eventsSF['modeBEAM_rh']=='car_hov3')|(eventsSF['modeBEAM_rh']=='car_hov2')|
                                              (eventsSF['modeBEAM_rh']=='hov2_teleportation')|(eventsSF['modeBEAM_rh']=='hov3_teleportation'), eventsSF['travelDistance'], np.nan)

In [51]:
eventsSF['duration_in_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                           (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                           (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                           (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                           (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['travelTime'], np.nan)

In [52]:
eventsSF['distance_transit'] = np.where((eventsSF['modeBEAM']=='bike_transit')|(eventsSF['modeBEAM']=='drive_transit')|
                                        (eventsSF['modeBEAM']=='walk_transit')|(eventsSF['modeBEAM']=='bus')|
                                        (eventsSF['modeBEAM']=='tram')|(eventsSF['modeBEAM']=='subway')|
                                        (eventsSF['modeBEAM']=='rail')|(eventsSF['modeBEAM']=='cable_car')|
                                        (eventsSF['modeBEAM']=='ride_hail_transit'), eventsSF['travelDistance'], np.nan)

In [53]:
# Removing the extra tour index happening after replanning events
eventsSF['replanningTime'] = np.where(eventsSF['type'] == 'Replanning', eventsSF['time'], np.nan)
eventsSF['replanningTime'] = eventsSF['replanningTime'].shift(+1)
eventsSF['tourIndex_fixed'] = np.where((eventsSF['type'] == 'ModeChoice')&(eventsSF['replanningTime'].notna()), np.nan, eventsSF['tourIndex'])

In [54]:
#eventsSF['fuelFood'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Food'), 
                                #eventsSF['primaryFuel'], np.nan)

In [56]:
#eventsSF['fuelElectricity'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Electricity'), 
                                #eventsSF['primaryFuel'], np.nan)

In [57]:
#eventsSF['fuelDiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Diesel'), 
                                #eventsSF['primaryFuel'], np.nan)

In [54]:
#eventsSF['fuelBiodiesel'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']=='Biodiesel'), 
                                #eventsSF['primaryFuel'], np.nan)

In [55]:
#eventsSF['fuel_not_Food'] = np.where((eventsSF['type']=='PathTraversal')&(eventsSF['primaryFuelType']!='Food')
                            #, eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [57]:
#eventsSF['fuelGasoline'] = np.where((eventsSF['type']=='PathTraversal')&((eventsSF['primaryFuelType']=='Gasoline')|(eventsSF['secondaryFuelType']=='Gasoline')), 
                           #eventsSF['primaryFuel']+eventsSF['secondaryFuel'], np.nan)

In [55]:
eventsSF['actEndType'] = np.where(eventsSF['type']=='actend', eventsSF['actType'], "")

In [56]:
eventsSF['actStartType'] = np.where(eventsSF['type']=='actstart', eventsSF['actType'], "")

In [53]:
#eventsSF[(eventsSF['modeBEAM_rh'] == 'ride_hail_pooled')|(eventsSF['type'] == 'Replanning')].head()

#### Trip Index

In [57]:
eventsSF["tripIndex"] = eventsSF.groupby("IDMerged")["tourIndex_fixed"].rank(method="first", ascending=True)
eventsSF["tripIndex"] = eventsSF.tripIndex.fillna(method='ffill')

In [139]:
eventsSF.duration_in_ridehail.sum()

69759545.0

In [134]:
a = eventsSF.describe()

In [136]:
a.to_csv('C:/Shared-Work/Data/CleanData/a.csv') 

In [150]:
eventsSF.head(200000).to_csv('C:/Shared-Work/Data/CleanData/BEAMCHECK.csv', index = False) 

#### Previous code for mode choice 

In [51]:
# Make a new column which determined mode choice numbers for each person and trip
#s = eventsSF.groupby(['IDMerged','tripIndex', 'type']).cumcount().add(1).astype(str).str.zfill(2)

In [52]:
#eventsSF['type_number'] = eventsSF['type'].astype(str) + s.astype(str)

In [53]:
#eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type_number'] == 'ModeChoice01' , eventsSF['modeBEAM_rh'], np.nan)

In [54]:
# Just showing the mode choice after replanning
#eventsSF['mode_choice_replanned_BEAM'] = np.where(eventsSF['type_number'] == 'ModeChoice02', eventsSF['modeBEAM_rh'], np.nan)

#### Only works if we have one replanning event

In [120]:
# Showing both mode choices before and after the replanning event
#eventsSF['mode_choice_act'] = eventsSF['mode_choice_replanned_BEAM'].combine_first(eventsSF['mode_choice_planned_BEAM'])

In [112]:
# If replanned, then the status is 1
#eventsSF['replanning_status'] = np.where(eventsSF['mode_choice_replanned_BEAM'].notna() , 1, np.nan)

In [113]:
# Group by and fill all the rows related to the replanning event
#eventsSF["replanning_status"] = eventsSF.groupby(['IDMerged','tripIndex'], as_index=False)['replanning_status'].transform(lambda x: x.ffill().bfill())

In [114]:
# Put all non-replanning events equal to 0
#eventsSF["replanning_status"] = np.where(eventsSF['replanning_status'].isna(), 0, eventsSF['replanning_status']).astype(int)

In [115]:
#eventsSF['mode_choice_actual_BEAM'] = np.where((eventsSF['replanning_status'] == 1) & (eventsSF['type_number']=='ModeChoice01'), np.nan, eventsSF['mode_choice_act'])

In [116]:
#eventsSF = eventsSF.drop(['mode_choice_act'], axis = 1)

In [117]:
#eventsSF[eventsSF['type'] == "Replanning"].tail(16)

In [118]:
#num_pass = eventsSF[eventsSF['IDMerged']==280]

In [62]:
#num_pass.to_csv('C:/Shared-Work/Data/CleanData/num_pass.csv', index = False)

In [63]:
#eventsSF["mode_choice_replanned_BEAM"] = eventsSF.groupby(['IDMerged','tripIndex'])['mode_choice_replanned_BEAM'].transform(lambda x: x.bfill())

In [65]:
#eventsSF['mode_choice_actual_BEAM'] = eventsSF['mode_choice_actual_BEAM'].combine_first(eventsSF['modeBEAM_rh'])

In [66]:
#eventsSF['mode_planned_tour_ActSim'] = np.where(eventsSF['type'] == 'PathTraversal', eventsSF['currentTourMode'], np.nan)

In [175]:
eventsSF.head(10000).to_csv('C:/Shared-Work/Data/CleanData/eventsNew_check.csv', index = False)   

In [98]:
#eventsSF['numPassengers_agg'] = eventsSF.groupby(['IDMerged','tripIndex'], as_index=False).numPassengers.agg(lambda x: ', '.join(set(x.dropna().astype(str))))

##### Mode Choice planned and actual

In [58]:
%%time
eventsSF['mode_choice_actual_BEAM'] = eventsSF.groupby(['IDMerged','tripIndex', 'type'])['modeBEAM'].transform('last')

Wall time: 12min 5s


In [59]:
%%time
eventsSF['mode_choice_planned_BEAM'] = eventsSF.groupby(['IDMerged','tripIndex', 'type'])['modeBEAM'].transform('first')

Wall time: 12min 2s


In [60]:
eventsSF['mode_choice_actual_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_actual_BEAM'])

In [61]:
eventsSF['mode_choice_planned_BEAM'] = np.where(eventsSF['type'] != 'ModeChoice' , np.nan, eventsSF['mode_choice_planned_BEAM'])

In [62]:
# Rename the "netCost" column
eventsSF.rename(columns={"netCost":"cost_BEAM"}, inplace=True) 

In [71]:
eventsSF[eventsSF['type']=='Replanning'].head()

,IDMerged,person,driver,riders,vehicle,time,type,modeBEAM,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,reason,link,legMode,facility,actType,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicleType,price,endY,endX,startY,startX,arrivalTime,departureTime,primaryFuelLevel,secondaryFuelLevel,cost,links,numPassengers,primaryFuel,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,departTime,score,incentive,tollCost,cost_BEAM,scenario,year,modeBEAM_rh,actEndTime,actStartTime,travelTime,travelDistance,length_mode_choice,duration_walking,distance_walking,duration_on_bike,distance_bike,duration_in_ridehail,distance_ridehail,duration_in_privateCar,distance_privateCar,duration_in_transit,distance_transit,replanningTime,tourIndex_fixed,actEndType,actStartType,tripIndex,mode_choice_actual_BEAM,mode_choice_planned_BEAM,replanning_status
5148,1004,1004,NaN,NaN,NaN,31368.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,NaN,NaN,1
5151,1004,1004,NaN,NaN,NaN,31374.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,NaN,NaN,1
5154,1004,1004,NaN,NaN,NaN,31380.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceCapacityExhausted WALK_TRANSIT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,NaN,NaN,1
9422,2192,2192,NaN,NaN,NaN,33300.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceUnavailable RIDE_HAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,3.0,NaN,NaN,1
9522,2192,2192,NaN,NaN,NaN,52200.0,Replanning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ResourceUnavailable RIDE_HAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,9.0,NaN,NaN,1


In [63]:
# Replanning events = 1, the rest = 0
eventsSF['replanning_status'] = np.where(eventsSF['type']=='Replanning', 1, 0)

In [64]:
#Change column type to string
eventsSF['reason'] = eventsSF['reason'].astype(str)

In [76]:
%%time
eventsSF['reason'].replace('nan', np.NaN)

Wall time: 1.27 s


KeyboardInterrupt: 

In [75]:
eventsSF.head()

,IDMerged,person,vehicle,time,type,modeBEAM,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,reason,link,legMode,facility,actType,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicleType,price,endY,endX,startY,startX,arrivalTime,departureTime,primaryFuelLevel,secondaryFuelLevel,cost,driver,links,numPassengers,primaryFuel,riders,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,departTime,score,incentive,tollCost,cost_BEAM,scenario,year,modeBEAM_rh,actEndTime,actStartTime,travelTime,travelDistance,length_mode_choice,duration_walking,distance_walking,duration_on_bike,distance_bike,duration_in_ridehail,distance_ridehail,duration_in_privateCar,distance_privateCar,duration_in_transit,distance_transit,replanningTime,tourIndex_fixed,actEndType,actStartType,tripIndex,mode_choice_actual_BEAM,mode_choice_planned_BEAM,replanning_status
0,1,1,NaN,29037.0,ModeChoice,hov3_teleportation,hov3_teleportation,NaN,HOV3_TELEPORTATION,56792.0,true,6498.758,1.0,"WALK,CAR_HOV3,WALK","body-1,teleportationSharedVehicle-1-1,body-1",Home,othmaint,577.0,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,hov3_teleportation,NaN,NaN,NaN,6498.758,6498.758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6498.758,NaN,NaN,NaN,1.0,,,1.0,hov3_teleportation,hov3_teleportation,0
1,1,1,NaN,29037.0,actend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,56792.0,NaN,NaN,Home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,29037.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home,,1.0,NaN,NaN,0
2,1,1,NaN,29037.0,departure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,56792.0,hov3_teleportation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,NaN,NaN,0
3,1,1,NaN,29601.0,TeleportationEvent,NaN,hov3_teleportation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.377254,-121.802441,37.350304,-121.80644,29601.0,29037.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,hov3_teleportation,NaN,NaN,564.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,564.0,NaN,NaN,NaN,NaN,NaN,,,1.0,NaN,NaN,0
4,1,1,NaN,29601.0,arrival,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,65790.0,car,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,NaN,NaN,0


In [149]:
eventsSF.reason.unique()

array(['nan', 'ResourceCapacityExhausted WALK_TRANSIT',
       'ResourceUnavailable RIDE_HAIL',
       'ResourceUnavailable RIDE_HAIL_POOLED',
       'ResourceCapacityExhausted DRIVE_TRANSIT',
       'MissedTransitPickup DRIVE_TRANSIT',
       'ResourceCapacityExhausted RIDE_HAIL_TRANSIT',
       'ResourceUnavailable RIDE_HAIL_TRANSIT',
       'MissedTransitPickup RIDE_HAIL_TRANSIT'], dtype=object)

In [ ]:
#eventsSF.loc['Total',:]= df.sum(axis=0)

In [55]:
eventsSF.type[eventsSF.type == 'Replanning'].count()

9606

In [159]:
Person_Trip_eventsSF.replanning_status[Person_Trip_eventsSF.replanning_status == 0].count()

2524052

In [66]:
%%time
Person_Trip_eventsSF = pd.pivot_table(
   eventsSF,
   index=['IDMerged','tripIndex'],
   aggfunc={'actStartTime': np.sum, 'actEndTime': np.sum, 'travelTime': np.sum, 'cost_BEAM': np.sum, 'actStartType': np.sum, 
            'actEndType': np.sum, 'duration_walking': np.sum, 'duration_in_privateCar': np.sum, 'duration_on_bike': np.sum, 
            'duration_in_ridehail': np.sum, 'travelDistance': np.sum, 'duration_in_transit': np.sum, 'distance_walking': np.sum, 
            'distance_bike': np.sum, 'distance_ridehail': np.sum, 'distance_privateCar': np.sum, 'distance_transit': np.sum, 
            'legVehicleIds': np.sum, 
            'mode_choice_planned_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))),
            'tripId': np.sum, 
            'vehicle': lambda x: ', '.join(set(x.dropna().astype(str))),
            'numPassengers': lambda x: ', '.join(list(x.dropna().astype(str))),
            'length_mode_choice': np.sum, 
            'replanning_status': np.sum,
            'reason': lambda x: ', '.join(list(x.dropna().astype(str)))}).reset_index()   

#'numPassengers': lambda x: ', '.join(set(x.dropna().astype(str))) 
#'mode_choice_actual_BEAM':lambda x: ', '.join(set(x.dropna().astype(str))) #
#'fuelFood': np.sum, 'fuelElectricity': np.sum, 'fuelBiodiesel': np.sum, 'fuelDiesel': np.sum, 'fuel_not_Food': np.sum, 'fuelGasoline': np.sum,
#'modeBEAM_rh': lambda x: ', '.join(list(x.dropna().astype(str))), 

Wall time: 19min 36s


In [67]:
Person_Trip_eventsSF['door_to_door_time'] = Person_Trip_eventsSF['actStartTime'] - Person_Trip_eventsSF['actEndTime'] 

In [68]:
Person_Trip_eventsSF['waitTime'] = Person_Trip_eventsSF['door_to_door_time'] - Person_Trip_eventsSF['travelTime'] 

In [69]:
Person_Trip_eventsSF['actPurpose'] = Person_Trip_eventsSF['actEndType'].astype(str) + "_to_" + Person_Trip_eventsSF['actStartType'].astype(str)

In [100]:
#Person_Trip_eventsSF.rename(columns={"modeBEAM_rh":"mode_BEAM_list"}, inplace=True) 

In [70]:
Person_Trip_eventsSF.rename(columns={"legVehicleIds":"legVehicleIds_estimate"}, inplace=True) 

In [71]:
Person_Trip_eventsSF.rename(columns={"vehicle":"vehicleIds"}, inplace=True) 

In [85]:
eventsSF[eventsSF['IDMerged'] == 253506]

,IDMerged,person,vehicle,time,type,modeBEAM,currentTourMode,expectedMaximumUtility,availableAlternatives,location,personalVehicleAvailable,length,tourIndex,legModes,legVehicleIds,currentActivity,nextActivity,tripId,reason,link,legMode,facility,actType,shiftStatus,parkingTaz,chargingPointType,pricingModel,parkingType,locationY,locationX,parkingZoneId,fuel,duration,vehicleType,price,endY,endX,startY,startX,arrivalTime,departureTime,primaryFuelLevel,secondaryFuelLevel,cost,driver,links,numPassengers,primaryFuel,riders,toStopIndex,fromStopIndex,seatingCapacity,tollPaid,capacity,linkTravelTime,secondaryFuel,secondaryFuelType,primaryFuelType,departTime,score,incentive,tollCost,cost_BEAM,scenario,year,modeBEAM_rh,actEndTime,actStartTime,travelTime,travelDistance,length_mode_choice,duration_walking,distance_walking,duration_on_bike,distance_bike,duration_in_ridehail,distance_ridehail,duration_in_privateCar,distance_privateCar,duration_in_transit,distance_transit,replanningTime,tourIndex_fixed,actEndType,actStartType,tripIndex,mode_choice_actual_BEAM,mode_choice_planned_BEAM,replanning_status
1273345,253506,253506,NaN,29127.0,ModeChoice,walk,walk,NaN,WALK,86010.0,false,60582.888120,1.0,WALK,body-253506,Home,work,83150153.0,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,walk,NaN,NaN,NaN,NaN,60582.888120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,,,1.0,walk,walk,0
1273346,253506,253506,NaN,29127.0,actend,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,86010.0,NaN,NaN,Home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,29127.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home,,1.0,NaN,NaN,0
1273347,253506,253506,NaN,29127.0,departure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,86010.0,walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,NaN,NaN,0
1273348,253506,253506,body-253506,29127.0,PersonEntersVehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,NaN,NaN,0
1273349,253506,NaN,body-253506,73028.0,PathTraversal,walk,walk,NaN,NaN,NaN,NaN,60582.888120,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BODY-TYPE-DEFAULT,NaN,37.833703,-122.275497,37.365645,-122.037758,73028.0,29127.0,2210000.000,0.0,NaN,253506,NaN,1.0,0.000,253506,NaN,NaN,0.0,0.0,0.0,NaN,0.0,None,Food,NaN,NaN,NaN,NaN,NaN,baseline,2018,walk,NaN,NaN,43901.0,60582.888120,NaN,43901.0,60582.888120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,NaN,NaN,0
1273350,253506,253506,body-253506,73028.0,PersonLeavesVehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,NaN,NaN,0
1273351,253506,253506,NaN,73028.0,arrival,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,114614.0,walk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,baseline,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,1.0,NaN,NaN,0
1273352,253506,253506,NaN,73028.0,

In [89]:
eventsSF.length.max()

340422.1206974114

In [83]:
b.to_csv('C:/Shared-Work/Data/CleanData/PivotTables/neg_time.csv', index = False)   

In [79]:
Person_Trip_eventsSF.door_to_door_time[Person_Trip_eventsSF.door_to_door_time<0].count()

154

In [86]:
a  = Person_Trip_eventsSF.describe()

In [88]:
a.to_csv('C:/Shared-Work/Data/CleanData/PivotTables/Person_Trip_eventsSF_describe.csv')   

In [116]:
sample_bike2 = eventsSF[(eventsSF['primaryFuelType'] == "Gasoline") & (eventsSF['modeBEAM'] == "bike") ]

In [74]:
Person_Trip_eventsSF.head(100000).to_csv('C:/Shared-Work/Data/CleanData/Person_Trip_eventsSF.csv', index = False)

#### Merging with activity sim persons and housholds files

In [104]:
actloc_2018_baseline = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-base-20220409/activitysim/"
actloc_2018_transitFreq05 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_0.5-20220228/activitysim/"
actloc_2018_transitFreq2 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-transit_frequencies_2.0-20220229/activitysim/"
actloc_2018_rhFleetsz0125 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_0.125-20220408/activitysim/"
actloc_2018_rhFleetsz175 = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-RH_fleetsz_1.75-20220408/activitysim/"

households = pd.read_csv(actloc_2018_baseline + 'final_households.csv')
persons = pd.read_csv(actloc_2018_baseline + 'final_persons.csv')
tours = pd.read_csv(actloc_2018_baseline +'final_tours.csv')
plans = pd.read_csv(actloc_2018_baseline +'final_plans.csv')
trips = pd.read_csv(actloc_2018_baseline + 'final_trips.csv')

In [129]:
persons.earning[persons.earning<0].count()

696

In [105]:
# Merge households and persons 
persons = persons.sort_values(by=['household_id'])
households = households.sort_values(by=['household_id'])
hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id')
#hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id', suffixes=('', '_drop'))
#hhpersons.drop([col for col in hhpersons.columns if 'drop' in col], axis=1, inplace=True)

In [106]:
# Merge tours, households and persons
tours = tours.sort_values(by=['person_id'])
hhpersons = hhpersons.sort_values(by=['person_id'])
hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id')
#hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id', suffixes=('', '_drop'))
#hhperTours.drop([col for col in hhperTours.columns if 'drop' in col], axis=1, inplace=True)

In [107]:
# Merge trips, tours, households and persons
trips = trips.sort_values(by=['person_id', 'tour_id'])
hhperTours = hhperTours.sort_values(by=['person_id','tour_id'])
tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'])
#tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'], suffixes=('', '_drop'))
#tourTripsMerged.drop([col for col in tourTripsMerged.columns if 'drop' in col], axis=1, inplace=True)

In [85]:
trips.head()

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode,mode_choice_logsum
507811,577,1,1735309,72,school,1,True,2,othmaint,595,641,5.730384,8.0,SHARED2FREE,-2.150332
507812,578,1,1735309,72,school,2,True,2,school,633,595,NaN,8.0,SHARED2FREE,-2.020166
507813,581,1,1735309,72,school,1,False,1,Home,641,633,NaN,14.0,SHARED2FREE,-1.999284
507814,689,2,1735309,86,atwork,1,True,1,atwork,700,700,NaN,12.0,DRIVEALONEFREE,-0.505729
507815,693,2,1735309,86,atwork,1,False,1,Work,700,700,NaN,14.0,DRIVEALONEPAY,-0.507800


In [108]:
# Merge person_trip level BEAM with activity sim merged files
tourTripsMerged = tourTripsMerged.sort_values(by=['person_id', 'trip_id'])
Person_Trip_eventsSF = Person_Trip_eventsSF.sort_values(by=['IDMerged','tripId'])
eventsASim = pd.merge(left=Person_Trip_eventsSF, right=tourTripsMerged, how='left', left_on=["IDMerged", 'tripId'], right_on=['person_id', 'trip_id'])
#eventsASim = pd.merge(left=Person_Trip_eventsSF, right=tourTripsMerged, how='left',left_on = ["IDMerged", 'tripId'] , right_on=['person_id', 'trip_id'], suffixes=('', '_drop'))
#eventsASim.drop([col for col in eventsASim.columns if 'drop' in col], axis=1, inplace=True)

In [109]:
eventsASim.rename(columns={"mode_choice_logsum_y":"logsum_tours_mode_AS_tours"}, inplace=True)

In [110]:
eventsASim.rename(columns={"tour_mode":"tour_mode_AS_tours"}, inplace=True)

In [111]:
eventsASim.rename(columns={"mode_choice_logsum_x":"logsum_trip_mode_AS_trips"}, inplace=True)

In [112]:
eventsASim.rename(columns={"trip_mode":"trip_mode_AS_trips"}, inplace=True)

In [113]:
eventsASim.head(10000).to_csv('C:/Shared-Work/Data/CleanData/Merged_SF_Baseline_2018.csv', index = False)

In [91]:
eventsASim.to_csv('C:/Shared-Work/Data/CleanData/ASim_BEAM_Merged_SF_Baseline_2018.csv', index = False)

In [98]:
eventsASim.duration_in_privateCar.sum()

1214953699.0

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(13, 6))
sns.heatmap(households, vmax=1, annot=True, linewidths=.5)
plt.xticks(rotation=30, horizontalalignment='right')
plt.show()

In [ ]:
import missingno as msno
%matplotlib inline
#msno.bar(SFbaseline2018)
#msno.heatmap(persons)

In [97]:
from io import StringIO # python3; python2: BytesIO 
import boto
import boto3

conn = boto.s3.connect_to_region('us-east-2')
bucket = conn.get_bucket('beam-core-act')
csv_buffer = StringIO()
eventsASim.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'eventsASim.csv').put(Body=csv_buffer.getvalue())

KeyboardInterrupt: 